<a href="https://colab.research.google.com/github/noushadkhan01/internshala/blob/master/internshala_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def clean_text(text):
  import string
  punctuations = string.punctuation.replace('-', '').replace("'", '') + '\n'
  text = [i for i in text if i not in punctuations]
  text = ''.join(text)
  text = text.strip()
  return text

In [0]:
def clean_stipend(text):
  import numpy as np
  text = text.lower().strip()
  text = [i for i in text]
  a = ''
  final_text = ''
  for i in text:
    a += i
    try:
      np.float64(a)
      final_text = a
    except:
      break
  if final_text:
    return np.float64(final_text)
  return np.nan

In [0]:
def clean_apply_by(text):
  f = text.find("'")
  text = text[:f] + ' 2020'
  if text == 'Not Provide 2020':
    return np.nan
  return text

In [0]:
def apply_by_to_date(text):
  from datetime import datetime
  a = datetime.strptime(text, '%d %b %Y')
  return a

In [0]:
def get_internshala_internships(url, n = 0):
  import numpy as np
  import pandas as pd
  import requests 
  from bs4 import BeautifulSoup
  all_items = []
  no_internship_text = ":( Sorry, We couldn't find internships matching your requirements."
  while True:
    print(f'n is {n}')
    if n:
      url = url + f'/page-{n}'
    try:
      html = requests.get(url)
      soup = BeautifulSoup(html.content, 'lxml')
      all_internship = soup.find_all('div', class_ = 'individual_internship')
      text = all_internship[0].find('h4').text
      if text == no_internship_text:
        print(no_internship_text)
        break
      l = len(all_internship)
      n2 = 0
      for i in all_internship:
        try:
          id = i.get('internshipid')
          location = i.find_all('a', class_ = 'location_link')[0].text
          link = 'https://internshala.com/' + i.find_all('a')[0].get('href')
          td = i.find_all('table')[0].find_all('td')
          start_date = td[0].text
          duration = td[1].text
          stipend = td[2].text
          posted_on = td[3].text
          apply_by = td[4].text
          items = [id, location, start_date, duration, stipend, posted_on, apply_by]
          items = list(map(clean_text, items))
          items.insert(2, link)
          del id, location, link, start_date, duration, stipend, posted_on, apply_by
          all_items.append(items)
          n2 += 1
        except:
          continue
      n += 1
    except Exception as e:
      print(f'{e.__class__.__name__}:--- {e.args[0]}')
      break
  if len(all_items) == 0:
    return no_internship_text
  df = pd.DataFrame(np.array(all_items), columns = ['id', 'location', 'link', 'start_date', 'duration', 'stipend', 'posted_on', 'apply_by'])
  df.drop_duplicates(inplace = True)
  df['stipend'] = df.stipend.apply(clean_stipend)
  df['apply_by'] = df['apply_by'].apply(clean_apply_by)
  df['apply_by'] = df['apply_by'].apply(apply_by_to_date)
  df.dropna(inplace = True)
  return df

In [0]:
url = 'https://internshala.com/internships/work-from-home-data%20science,machine%20learning-jobs'

In [9]:
work_from_home = get_internshala_internships(url)

n is 0
n is 1
n is 2
n is 3
n is 4
:( Sorry, We couldn't find internships matching your requirements.


In [10]:
work_from_home.head()

,id,location,link,start_date,duration,stipend,posted_on,apply_by
0,886861,Work From Home,https://internshala.com//internship/details/de...,Immediately,6 Months,2000.0,26 Mar'20,2020-04-23
1,884133,Work From Home,https://internshala.com//internship/detail/dat...,20 Apr - 30 Apr'20,4 Months,1000.0,20 Mar'20,2020-04-17
2,896796,Work From Home,https://internshala.com//internship/detail/mac...,Immediately,3 Months,3000.0,17 Apr'20,2020-05-15
3,896591,Work From Home,https://internshala.com//internship/detail/pyt...,Immediately,3 Months,10000.0,17 Apr'20,2020-05-15
4,895918,Work From Home,https://internshala.com//internship/detail/dat...,Immediately,3 Months,9000.0,16 Apr'20,2020-05-14


In [11]:
type(work_from_home.apply_by.values[0])

numpy.datetime64

In [12]:
work_from_home.shape

(83, 8)

In [0]:
work_from_home.sort_values(by = 'apply_by', inplace = True)

In [14]:
work_from_home.head(2)

,id,location,link,start_date,duration,stipend,posted_on,apply_by
1,884133,Work From Home,https://internshala.com//internship/detail/dat...,20 Apr - 30 Apr'20,4 Months,1000.0,20 Mar'20,2020-04-17
121,884602,Work From Home,https://internshala.com//internship/detail/act...,Immediately,4 Months,5000.0,20 Mar'20,2020-04-17


In [22]:
useful_data = work_from_home[work_from_home.apply_by <= '2020-04-25']
useful_data.shape

(14, 8)

In [0]:
#useful_data.sort_values(by = 'stipend', ascending = False, inplace = True)

In [24]:
useful_data

,id,location,link,start_date,duration,stipend,posted_on,apply_by
1,884133,Work From Home,https://internshala.com//internship/detail/dat...,20 Apr - 30 Apr'20,4 Months,1000.0,20 Mar'20,2020-04-17
121,884602,Work From Home,https://internshala.com//internship/detail/act...,Immediately,4 Months,5000.0,20 Mar'20,2020-04-17
122,884098,Work From Home,https://internshala.com//internship/details/de...,Immediately,1 Month,10000.0,20 Mar'20,2020-04-18
118,884829,Work From Home,https://internshala.com//internship/details/da...,Immediately,3 Months,6000.0,23 Mar'20,2020-04-21
117,885390,Work From Home,https://internshala.com//internship/details/bl...,Immediately,3 Months,6000.0,23 Mar'20,2020-04-21
116,885696,Work From Home,https://internshala.com//internship/details/na...,Immediately,2 Months,5000.0,23 Mar'20,2020-04-21
115,886271,Work From Home,https://internshala.com//internship/details/ma...,Immediately,1 Month,2000.0,24 Mar'20,2020-04-21
0,886861,Work From Home,https://internshala.com//internship/details/de...,Immediately,6 Months,2000.0,26 Mar'20,2020-04-23
110,887115,Work From Home,https://internshala.com//internship/details/re...,Immediately,1 Month,5000.0,26 Mar'20,2020-04-23
113,886292,Work From Home,https://internshala.com//internship/details/da...,Immediately,6 Months,5000.0,25 Mar'20,2020-04-23


In [25]:
print(useful_data[['link', 'duration', 'stipend', 'apply_by']].values)

[['https://internshala.com//internship/detail/data-analytics-work-from-home-job-internship-at-motilal-oswal1584672410'
  '4 Months' 1000.0 Timestamp('2020-04-17 00:00:00')]
 ['https://internshala.com//internship/detail/actuary-or-actuarial-intern-work-from-home-job-internship-at-get-ria1584702655'
  '4 Months' 5000.0 Timestamp('2020-04-17 00:00:00')]
 ['https://internshala.com//internship/details/deep-learning-work-from-home-job-internship-at-medi-caps-university1584671179'
  '1 Month' 10000.0 Timestamp('2020-04-18 00:00:00')]
 ['https://internshala.com//internship/details/data-science-content-development-work-from-home-job-internship-at-dphi1584722417'
  '3 Months' 6000.0 Timestamp('2020-04-21 00:00:00')]
 ['https://internshala.com//internship/details/blockchain-analytics-work-from-home-job-internship-at-buyucoin1584951851'
  '3 Months' 6000.0 Timestamp('2020-04-21 00:00:00')]
 ['https://internshala.com//internship/details/natural-language-processing-work-from-home-job-internship-at-a